In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint

# RepVGG

<img src="https://i.ibb.co/wrxq7Kv/image.png" alt="image" border="0">

## Reparameterization

Chứng minh kỹ thuật reparameterization, gọi đầu vào hình ảnh $X \in \mathbb{R}^{n \times n}$ là một ma trận có dạng:

$$ X = 
\begin{pmatrix}
x_{11} & x_{12} & x_{13} & x_{14} & x_{15} \\
x_{21} & x_{22} & x_{23} & x_{24} & x_{25} \\ 
x_{31} & x_{32} & x_{33} & x_{34} & x_{35} \\
x_{41} & x_{42} & x_{44} & x_{44} & x_{45} \\
x_{51} & x_{52} & x_{55} & x_{55} & x_{55} \\
\end{pmatrix}
$$

và ma trận trọng số $W$ thực hiện convolution:

$$ W = 
\begin{pmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} \\
w_{31} & w_{32} & w_{33} \\
\end{pmatrix}
$$

Gọi ma trận kết quả sau khi thực hiện phép convolution (padding=$1$) $X * W$ là $Y$:

$$ Y =
\begin{pmatrix}
y_{11} & y_{12} & y_{13} & y_{14} & y_{15} \\
y_{21} & y_{22} & y_{23} & y_{24} & y_{25} \\ 
y_{31} & y_{32} & y_{33} & y_{34} & y_{35} \\
y_{41} & y_{42} & y_{44} & y_{44} & y_{45} \\
y_{51} & y_{52} & y_{55} & y_{55} & y_{55} \\
\end{pmatrix}
$$

Trong đó $y_{22} = x_{11} \times w_{11} + x_{12} \times w_{12} + \dots + x_{33} \times w_{33}$

Gọi ma trận kết quả sau khi thực hiện phép convolution pointwise với giá trị $p$ là $Z$:
$$ Z =
\begin{pmatrix}
z_{11} & z_{12} & z_{13} & z_{14} & z_{15} \\
z_{21} & z_{22} & z_{23} & z_{24} & z_{25} \\ 
z_{31} & z_{32} & z_{33} & z_{34} & z_{35} \\
z_{41} & z_{42} & z_{44} & z_{44} & z_{45} \\
z_{51} & z_{52} & z_{55} & z_{55} & z_{55} \\
\end{pmatrix}
$$

Trong đó $z_{22} = x_{22} \times p$
Vậy trong ma trận kết quả $K = Y + Z$ thì vị trí $K_{22} = y_{22} + z_{22} = (x_{11} \times w_{11} + x_{12} \times w_{12} + \dots + x_{33} \times w_{33}) + (x_{22} \times p)$


Đây là operations diễn ra trong lúc train. Tuy nhiên khi inference thì ta áp dụng kỹ thuật reparameterization để giảm lượng params tính toán hay FLOPS bằng các bước sau:
1. Padding giá trị $p$ thành ma trận trọng số $P$ có shape bằng ma trận $W$
2. Tạo ma trận trọng số mởi là $E = W + E$
3. Thực hiện phép convolution $X * E$ 

Lúc này ma trận được padded $P$ có dạng
$$ P = 
\begin{pmatrix}
0 & 0 & 0 \\
0 & p & 0 \\
0 & 0 & 0 \\
\end{pmatrix}
$$

Ma trận trọng số mới $E$ có dạng:
$$ E = 
\begin{pmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} + p & w_{23} \\
w_{31} & w_{32} & w_{33} \\
\end{pmatrix}
$$

Ma trận kết quả $K'$ có thành phần $K'_{22} = x_{11} \times w_{11} + x_{12} \times w_{12} + \dots + (w_{22} + p) \times x_{22} + \dots + x_{33} \times w_{33} $. \
Chú ý phần tử $(w_{22} + p) \times x_{22} = w_{22} \times x_{22} + x_{22} \times p$
Từ đây ta dễ dàng so sánh và thấy $K = K'$

In [2]:
X = torch.tensor([
    [1, 2, 3, 4, 5],
    [2, 3, 5, 1, 2],
    [9, 8, 1, 5, 4],
    [0, 1, 2, 4, 0],
    [5, 6, 7, 8, 9],
]).unsqueeze(0).to(torch.float32)

W = torch.tensor([
    [0.5, 0.1, 0.2],
    [0.2, 0.3, 0.9],
    [0.1, 0.4, 0.6],
]).unsqueeze(0).unsqueeze(0)

P = torch.tensor([
    [0.7]
]).unsqueeze(0).unsqueeze(0)

Y = F.conv2d(X, W, stride=1, padding=1)
Z = F.conv2d(X, P, stride=1, padding=0)
K = Y + Z
K

tensor([[[ 5.4000,  9.3000,  9.9000, 11.2000,  6.7000],
         [13.6000, 13.9000, 12.8000, 11.2000,  6.8000],
         [17.6000, 14.6000, 12.6000, 13.6000,  6.1000],
         [ 9.0000, 15.4000, 19.1000, 15.5000,  8.1000],
         [10.6000, 13.8000, 16.9000, 18.9000, 12.6000]]])

In [3]:
X = torch.tensor([
    [1, 2, 3, 4, 5],
    [2, 3, 5, 1, 2],
    [9, 8, 1, 5, 4],
    [0, 1, 2, 4, 0],
    [5, 6, 7, 8, 9],
]).unsqueeze(0).to(torch.float32)

W = torch.tensor([
    [0.5, 0.1, 0.2],
    [0.2, 0.3, 0.9],
    [0.1, 0.4, 0.6],
]).unsqueeze(0).unsqueeze(0)

P = torch.tensor([
    [0., 0., 0.],
    [0., 0.7, 0.],
    [0., 0., 0.],
]).unsqueeze(0).unsqueeze(0)

E = W + P

K_ = F.conv2d(X, E, stride=1, padding=1)
K_

tensor([[[ 5.4000,  9.3000,  9.9000, 11.2000,  6.7000],
         [13.6000, 13.9000, 12.8000, 11.2000,  6.8000],
         [17.6000, 14.6000, 12.6000, 13.6000,  6.1000],
         [ 9.0000, 15.4000, 19.1000, 15.5000,  8.1000],
         [10.6000, 13.8000, 16.9000, 18.9000, 12.6000]]])

### How BatchNorm works

Read more in this paper [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)

Cho input $X \in \mathbb{R}^{B \times C \times H \times W}$ với $B,C,H,W$ lần lượt là batch size, channels, height, width. Khi đưa qua BatchNorm, thì $\text{BN}(X)$ được tính bằng cách: \
- Khởi tạo vector $\mu$ (mean) và $v$ (variance) theo phân phối chuẩn:
$$
\mu, v\in \mathbb{R}^C \space | \space  \mu_i=0, v_i=1 \space | \space \forall \mu_i \in \mu, \forall v_i \in v
$$

- Sau đó $X$ sẽ được biến đối trong không gian bằng 2 phép $\text{normalize}$ và $\text{scale+shift}$

$$
X_{\text{norm}} = \frac{X - \mu}{\sqrt{v + \epsilon}} \\
X_{\text{new}} = X_{\text{norm}} \gamma + \beta
$$

- Trong đó epsilon $\epsilon$ được khởi tạo $=1e^{-5}$ và $\gamma, \beta \in R^C$ hay là vector shape là $(1, C, 1, 1)$. Hay từng feature $X_{\text{new}}[:,i:,:]$ trong $X$ được tính:

$$
X^{\text{new}}_{[:,i:,:]} =  \frac{X_{[:,i,:,:]} - \mu_i}{\sqrt{v + \epsilon}} \gamma + \beta_i
$$

## BatchNorm kết hợp Convolution
Như ta đã biết, tổ hợp phổ biến trong computer vision là $\text{Conv-Relu-BatchNorm}$. BatchNorm diễn ra sau Convolution (ví dụ bỏ qua ReLU) thì làm sao ta kết hợp bước Conv-BatchNorm trong lúc inference để tính toán nhanh hơn.  Gọi $X \in \mathbb{R}^{B, C_{in}, H, W}$ là ma trận đầu vào $W \in \mathbb{W}^{C_{out}, C_{in}, K, K}$ là ma trận trọng số để thực hiện phép convolution và ma trận kết quả là $Y = X * W + b \space | \space Y \in \mathbb{R}^{B, C_{out}, H', W'}$ với $*$ là phép convolution và $b$ là bias.

$$
\text{BN}(Y, \mu, v, \gamma, \beta) = (Y - \mu) \frac{\gamma}{\sqrt{v + \epsilon}} + \beta \\
= (X * W + b - \mu) \frac{\gamma}{\sqrt{v + \epsilon}}. \\
= X * (W \frac{\gamma}{\sqrt{v + \epsilon}}) + \frac{\gamma}{\sqrt{v + \epsilon}} (b - \mu) + \beta
= X * W' + b'
$$

Vậy ma trận trọng số mới là $W' = W \frac{\gamma}{\sqrt{v + \epsilon}}$ và bias mới là $b'= \frac{\gamma}{\sqrt{v + \epsilon}} (b - \mu) + \beta$. \\
Nhưng ở đây ta cần phải lưu ý thêm về chiều khi nhần vào của $W'_{[i,:,:,:]} = \frac{\gamma_i}{\sqrt{v_i + \epsilon}} W_{[i,:,:,:]}$. Đây là lý do ta phải reshape các vector $\gamma, v \in (1, C_{out}, 1, 1)$ thành $(C_{out}), 1, 1, 1)$ trước khi nhân.


In [4]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, eps=0, momentum=0.1, training_mode=False):
        super().__init__()

        self.training_mode = training_mode
        self.momentum = 0.1
        self.eps = eps

        # trainable parameters
        self.gamma = nn.Parameter(torch.ones(1, num_features, 1, 1))
        self.beta = nn.Parameter(torch.zeros(1, num_features, 1, 1))

        # running mean & variance
        self.r_mean = torch.zeros(1, num_features, 1, 1)
        self.r_var = torch.ones(1, num_features, 1, 1)

    def forward(self, x):
        if self.training_mode:
            x_mean = x.mean([0, 2, 3], keepdim=True)
            x_var = x.var([0, 2, 3], keepdim=True, unbiased=False)

            # Update running mean and variance
            self.r_mean = (1 - self.momentum) * self.r_mean + self.momentum * x_mean
            self.r_var = (1 - self.momentum) * self.r_var + self.momentum * x_var

        else:
            x_mean = self.r_mean
            x_var = self.r_var

        x_norm = (x - x_mean) / torch.sqrt(x_var + self.eps)         # Normalize
        x_out = x_norm * self.gamma + self.beta                      # Scale and Shift
        return x_out

In [ ]:
# Initialize matrix X, weight W and bias b
X = torch.randn(12, 32, 224, 224)
W = torch.randn(64, 32, 3, 3)
b = torch.randn(64)

In [ ]:
# Normal convolution
Y = F.conv2d(X, W, b, stride=1, padding=1)
bn = BatchNorm(64)
Z = bn(Y)

In [ ]:
# Reshape before multiplication
gamma = bn.gamma.view(64, 1, 1, 1)
var = bn.r_var.view(64, 1, 1, 1)
mean = bn.r_mean.view(64, 1, 1, 1)
beta = bn.beta.view(64, 1, 1, 1)
eps = bn.eps

In [ ]:
# Reparameterization
W_ = W * (gamma / torch.sqrt(var + eps))
b_ = (gamma / torch.sqrt(var + eps)) * (b.view(64, 1, 1, 1) - mean) + beta
b_ = b_.squeeze()

Z_ = F.conv2d(X, W_, b_, stride=1, padding=1)

In [ ]:
# Testing
print(Z[2,10,56,56])
print(Z_[2,10,56,56])

### RepVGG Implementation